# Create PostgreSQL Database of NCAA Men's Basketball Historic Data
![](http://localhost:8888/tree/00_public/000_Git_Tracked_Notebooks/Git_Tracked_Notebooks/phx-final-four-logo.jpg)

#### Imports & Versions

In [1]:
import ujson
print("ujson v.{}".format(ujson.__version__))

import pandas as pd
print("pandas v.{}".format(pd.__version__))

import sqlalchemy
print("SqlAlchemy v.{}".format(sqlalchemy.__version__))

ujson v.1.35
pandas v.0.19.2
SqlAlchemy v.1.1.5


In [6]:
#Path to .csv files that contain the data to be added
file_path = "D:\\_data_science_data\\2017_march_madness\\tourney17\\"

### Open PostgreSQL Credentials from local file

In [2]:
credentials = ujson.load(open("D:\\Python\\posgresql_credentials.json","r"))

In [3]:
#availible databases
credentials['databases']

['temp_db_rental', 'ncaa_mbb']

### Define Function to Connect to Database

In [26]:
def pg_connect(user, password, db, host='localhost',port=5432):
    '''Returns a connection and metada SqlAlchemy Object'''
    url = 'postgresql://{}:{}@{}:{}/{}'
    url = url.format(user, password, host, port, db)
    engine = sqlalchemy.create_engine(url, client_encoding = 'utf8')
    meta = sqlalchemy.MetaData()#bind = con, reflect = True)
    
    return (engine, meta)

### Create the engine/con, meta and connection objects

In [27]:
engine, meta = pg_connect(credentials['user'],
                       credentials['password'],
                       credentials['databases'][1])
connection = engine.connect()

### Create Blank Table for Teams

In [13]:
#Create Team Table (blank)
#teams = sqlalchemy.Table('teams', meta,
#                        sqlalchemy.Column('team_id', sqlalchemy.String, primary_key = True),
#                        sqlalchemy.Column('team_name', sqlalchemy.String))
#meta.create_all(engine)

In [14]:
#ensure the table was created
#print(engine.table_names())

['teams']


### Create Blank Table for Seeds

In [15]:
#Create Tournament Seed table (blank)
#tourney_seeds = sqlalchemy.Table('tourney_seeds',meta,
#                                 sqlalchemy.Column('season', sqlalchemy.String),
#                                 sqlalchemy.Column('seed', sqlalchemy.String),
#                                 sqlalchemy.Column('team',sqlalchemy.String, sqlalchemy.ForeignKey('teams.team_id')))
#meta.create_all(engine)

In [11]:
#print(engine.table_names())

['teams', 'tourney_seeds']


### Pull Team Data into Dataframe from local .csv File

In [24]:
#Load Teams dataframe
#teams_df = pd.read_csv(file_path+'Teams.csv')
#teams_df = teams_df.rename(index = str, columns = {'Team_Id':'team_id', 'Team_Name':'team_name'})
#teams_df.head()

,team_id,team_name
0,1101,Abilene Chr
1,1102,Air Force
2,1103,Akron
3,1104,Alabama
4,1105,Alabama A&M


### Append Data from Teams DataFrame to existing PostgreSQL Table

In [25]:
#insert data into existing table by appending it to the end
#teams_df.to_sql('teams', engine, if_exists = 'append', index = False)

In [26]:
#Validate insertion by running select statemnt against table
stmt = 'SELECT * FROM teams LIMIT 6'
res_proxy = connection.execute(stmt)
results = res_proxy.fetchall()
for res in results:
    print(res)

('1101', 'Abilene Chr')
('1102', 'Air Force')
('1103', 'Akron')
('1104', 'Alabama')
('1105', 'Alabama A&M')
('1106', 'Alabama St')


### Pull Seed Data into DataFrame from local .csv File

In [29]:
#Load Seeds dataframe
#seeds_df = pd.read_csv(file_path+'TourneySeeds.csv')
#seeds_df = seeds_df.rename(index = str, columns = {'Season':'season', 'Seed':'seed', 'Team':'team'})
#seeds_df.head()

,season,seed,team
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


### Append Data from Seed DataFrame to existing PostgreSQL Table

In [30]:
#insert data into existing table by appending it to the end
#seeds_df.to_sql('tourney_seeds', engine, if_exists = 'append', index = False)

In [31]:
#Validate insertion by running select statemnt against table
stmt = 'SELECT * FROM tourney_seeds LIMIT 6'
res_proxy = connection.execute(stmt)
results = res_proxy.fetchall()
for res in results:
    print(res)

('1985', 'W01', '1207')
('1985', 'W02', '1210')
('1985', 'W03', '1228')
('1985', 'W04', '1260')
('1985', 'W05', '1374')
('1985', 'W06', '1208')


### Validate Keys in Table(s) by pulling a Select with a join

In [32]:
#Validate insertion by running select statemnt against table
stmt = 'SELECT tourney_seeds.season, tourney_seeds.seed, teams.team_name \
        FROM tourney_seeds \
        INNER JOIN teams \
        ON tourney_seeds.team=teams.team_id \
        LIMIT 6'
res_proxy = connection.execute(stmt)
results = res_proxy.fetchall()
for res in results:
    print(res)

('1985', 'W01', 'Georgetown')
('1985', 'W02', 'Georgia Tech')
('1985', 'W03', 'Illinois')
('1985', 'W04', 'Loyola-Chicago')
('1985', 'W05', 'SMU')
('1985', 'W06', 'Georgia')


### Convert Select Statement Results into DataFrame

In [6]:
stmt = 'SELECT tourney_seeds.season, tourney_seeds.seed, teams.team_name \
        FROM tourney_seeds \
        INNER JOIN teams \
        ON tourney_seeds.team=teams.team_id'

res_proxy = connection.execute(stmt)

query_df = pd.DataFrame(res_proxy.fetchall())
query_df.columns = res_proxy.keys()
query_df.head()

,season,seed,team_name
0,1985,W01,Georgetown
1,1985,W02,Georgia Tech
2,1985,W03,Illinois
3,1985,W04,Loyola-Chicago
4,1985,W05,SMU


### Add remaining .CSV files to PostgreSQL Database as is

In [10]:
#TeamConferences_Thru2017.csv - season, team_id, conference
#conferences_df = pd.read_csv(file_path+'TeamConferences_Thru2017.csv')
#conferences_df.head()

,season,team_id,conference
0,1985,1114,a_sun
1,1985,1147,a_sun
2,1985,1204,a_sun
3,1985,1209,a_sun
4,1985,1215,a_sun


In [8]:
#team_conferences = sqlalchemy.Table('team_conferences',meta,
#                                 sqlalchemy.Column('season', sqlalchemy.String),
#                                 sqlalchemy.Column('team_id', sqlalchemy.String),
#                                 sqlalchemy.Column('conference',sqlalchemy.String))
#meta.create_all(engine)

In [11]:
#conferences_df.to_sql('team_conferences', engine, if_exists = 'append', index = False)

In [13]:
stmt = 'SELECT * FROM team_conferences LIMIT 5'

res_proxy = connection.execute(stmt)

query_df = pd.DataFrame(res_proxy.fetchall())
query_df.columns = res_proxy.keys()
query_df

,season,team_id,conference
0,1985,1114,a_sun
1,1985,1147,a_sun
2,1985,1204,a_sun
3,1985,1209,a_sun
4,1985,1215,a_sun


In [16]:
#TourneyGeog.csv - season, daynum, wteam, lteam, host, lat, long
#tgeog_df = pd.read_csv(file_path+'TourneyGeog.csv')
#tgeog_df = tgeog_df.rename(index = str, columns = {'lng':'lon'})
#tgeog_df.head()

,season,daynum,wteam,lteam,slot,host,lat,lon
0,1985,136,1207,1250,R1W1,hartford,41.7627,-72.6743
1,1985,136,1260,1233,R1W4,hartford,41.7627,-72.6743
2,1985,136,1374,1330,R1W5,hartford,41.7627,-72.6743
3,1985,136,1396,1439,R1W8,hartford,41.7627,-72.6743
4,1985,136,1385,1380,R1X1,tulsa,36.1314,-95.9372


In [18]:
#tourney_geography = sqlalchemy.Table('tourney_geography',meta,
#                                     sqlalchemy.Column('season', sqlalchemy.String),
#                                     sqlalchemy.Column('daynum', sqlalchemy.String),
#                                     sqlalchemy.Column('wteam', sqlalchemy.String),
#                                     sqlalchemy.Column('lteam', sqlalchemy.String),
#                                     sqlalchemy.Column('slot', sqlalchemy.String),
#                                     sqlalchemy.Column('host', sqlalchemy.String),
#                                     sqlalchemy.Column('lat',sqlalchemy.Float),
#                                     sqlalchemy.Column('lon',sqlalchemy.Float))
#meta.create_all(engine)

In [19]:
#tgeog_df.to_sql('tourney_geography', engine, if_exists = 'append', index = False)

In [20]:
stmt = 'SELECT * FROM tourney_geography LIMIT 5'

res_proxy = connection.execute(stmt)

query_df = pd.DataFrame(res_proxy.fetchall())
query_df.columns = res_proxy.keys()
query_df

,season,daynum,wteam,lteam,slot,host,lat,lon
0,1985,136,1207,1250,R1W1,hartford,41.7627,-72.6743
1,1985,136,1260,1233,R1W4,hartford,41.7627,-72.6743
2,1985,136,1374,1330,R1W5,hartford,41.7627,-72.6743
3,1985,136,1396,1439,R1W8,hartford,41.7627,-72.6743
4,1985,136,1385,1380,R1X1,tulsa,36.1314,-95.9372


### Restructure Season Game Details

In [21]:
#RegularSeasonDetailedResults.csv
#rsdr_df = pd.read_csv(file_path+'RegularSeasonDetailedResults.csv')

In [23]:
#rows = []
#for x in range(len(rsdr_df)):
#    temp_tr = list(rsdr_df.iloc[x])
#    if temp_tr[6] == 'H':
#        loc = [1,0,0]
#    elif temp_tr[6] == 'A':
#        loc = [0,1,0]
#    else:
#        loc = [0,0,1]
#        
#    rows.append([temp_tr[0],temp_tr[1],temp_tr[2],1,temp_tr[4],loc[0],loc[2],temp_tr[7],temp_tr[3],temp_tr[8],
#                          temp_tr[9],temp_tr[10],temp_tr[11],temp_tr[12],temp_tr[13],temp_tr[14],temp_tr[15],temp_tr[16],
#                          temp_tr[17],temp_tr[18],temp_tr[19],temp_tr[20],temp_tr[33],temp_tr[5],temp_tr[21],temp_tr[22],
#                          temp_tr[23],temp_tr[24],temp_tr[25],temp_tr[26],temp_tr[27],temp_tr[28],temp_tr[29],temp_tr[30],
#                          temp_tr[31],temp_tr[32]])
#    rows.append([temp_tr[0],temp_tr[1],temp_tr[4],0,temp_tr[2],loc[1],loc[2],temp_tr[7],temp_tr[5],temp_tr[21],
#                          temp_tr[22],temp_tr[23],temp_tr[24],temp_tr[25],temp_tr[26],temp_tr[27],temp_tr[28],temp_tr[29],
#                          temp_tr[30],temp_tr[31],temp_tr[32],temp_tr[33],temp_tr[20],temp_tr[3],temp_tr[8],temp_tr[9],
#                          temp_tr[10],temp_tr[11],temp_tr[12],temp_tr[13],temp_tr[14],temp_tr[15],temp_tr[16],temp_tr[17],
#                          temp_tr[18],temp_tr[19]])
#print(len(rows))

153272


In [24]:
#rs_games_df = pd.DataFrame(rows, columns = ['season','daynum','team','win','opponent','home_game','neutral_location','num_ot','score',
#                                      'fgm','fga','fgm3','fga3','ftm','fta','or','dr','ast','to','stl','blk','fouls_given',
#                                      'fouls_received','opp_score','opp_fgm','opp_fga','opp_fgm3','opp_fga3','opp_ftm','opp_fta',
#                                      'opp_or','opp_dr','opp_ast','opp_to','opp_stl','opp_blk'])

In [29]:
#print(len(rs_games_df))
#rs_games_df.head()

153272


,season,daynum,team,win,opponent,home_game,neutral_location,num_ot,score,fgm,...,opp_fgm3,opp_fga3,opp_ftm,opp_fta,opp_or,opp_dr,opp_ast,opp_to,opp_stl,opp_blk
0,2003,10,1104,1,1328,0,1,0,68,27,...,2,10,16,22,10,22,8,18,9,2
1,2003,10,1328,0,1104,0,1,0,62,22,...,3,14,11,18,14,24,13,23,7,1
2,2003,10,1272,1,1393,0,1,0,70,26,...,6,24,9,20,20,25,7,12,8,6
3,2003,10,1393,0,1272,0,1,0,63,24,...,8,20,10,19,15,28,16,13,4,4
4,2003,11,1266,1,1437,0,1,0,73,24,...,3,26,14,23,31,22,9,12,2,5


In [30]:
#season_games = sqlalchemy.Table('season_games',meta,
#                                sqlalchemy.Column('season', sqlalchemy.String),
#                                sqlalchemy.Column('daynum', sqlalchemy.String),
#                                sqlalchemy.Column('team', sqlalchemy.String),
#                                sqlalchemy.Column('win', sqlalchemy.Integer),
#                                sqlalchemy.Column('opponent', sqlalchemy.String),
#                                sqlalchemy.Column('home_game', sqlalchemy.Integer),
#                                sqlalchemy.Column('neutral_location', sqlalchemy.Integer),
#                                sqlalchemy.Column('num_ot', sqlalchemy.Integer),
#                                sqlalchemy.Column('score', sqlalchemy.Integer),
#                                sqlalchemy.Column('fgm', sqlalchemy.Integer),
#                                sqlalchemy.Column('fga', sqlalchemy.Integer),
#                                sqlalchemy.Column('fgm3', sqlalchemy.Integer),
#                                sqlalchemy.Column('fga3', sqlalchemy.Integer),
#                                sqlalchemy.Column('ftm', sqlalchemy.Integer),
#                                sqlalchemy.Column('fta', sqlalchemy.Integer),
#                                sqlalchemy.Column('or', sqlalchemy.Integer),
#                                sqlalchemy.Column('dr', sqlalchemy.Integer),
#                                sqlalchemy.Column('ast', sqlalchemy.Integer),
#                                sqlalchemy.Column('to', sqlalchemy.Integer),
#                                sqlalchemy.Column('stl', sqlalchemy.Integer),
#                                sqlalchemy.Column('blk', sqlalchemy.Integer),
#                                sqlalchemy.Column('fouls_given', sqlalchemy.Integer),
#                                sqlalchemy.Column('fouls_received', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_score', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_fgm', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_fga', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_fgm3', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_fga3', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_ftm', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_fta', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_or', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_dr', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_ast', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_to', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_stl', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_blk', sqlalchemy.Integer))
#meta.create_all(engine)

In [31]:
#rs_games_df.to_sql('season_games', engine, if_exists = 'append', index = False)

In [33]:
stmt = 'SELECT * FROM season_games LIMIT 5'

res_proxy = connection.execute(stmt)

query_df = pd.DataFrame(res_proxy.fetchall())
query_df.columns = res_proxy.keys()
query_df

,season,daynum,team,win,opponent,home_game,neutral_location,num_ot,score,fgm,...,opp_fgm3,opp_fga3,opp_ftm,opp_fta,opp_or,opp_dr,opp_ast,opp_to,opp_stl,opp_blk
0,2003,10,1104,1,1328,0,1,0,68,27,...,2,10,16,22,10,22,8,18,9,2
1,2003,10,1328,0,1104,0,1,0,62,22,...,3,14,11,18,14,24,13,23,7,1
2,2003,10,1272,1,1393,0,1,0,70,26,...,6,24,9,20,20,25,7,12,8,6
3,2003,10,1393,0,1272,0,1,0,63,24,...,8,20,10,19,15,28,16,13,4,4
4,2003,11,1266,1,1437,0,1,0,73,24,...,3,26,14,23,31,22,9,12,2,5


### Restructure Tournament Game Details

In [42]:
#TourneyDetailedResults.csv
#tgdr_df = pd.read_csv(file_path+'TourneyDetailedResults.csv')

In [35]:
#rows = []
#for x in range(len(tgdr_df)):
#    temp_tr = list(tgdr_df.iloc[x])
#    rows.append([temp_tr[0],temp_tr[1],temp_tr[2],1,temp_tr[4],temp_tr[7],temp_tr[3],temp_tr[8],
#                          temp_tr[9],temp_tr[10],temp_tr[11],temp_tr[12],temp_tr[13],temp_tr[14],temp_tr[15],temp_tr[16],
#                          temp_tr[17],temp_tr[18],temp_tr[19],temp_tr[20],temp_tr[33],temp_tr[5],temp_tr[21],temp_tr[22],
#                          temp_tr[23],temp_tr[24],temp_tr[25],temp_tr[26],temp_tr[27],temp_tr[28],temp_tr[29],temp_tr[30],
#                          temp_tr[31],temp_tr[32]])
#    rows.append([temp_tr[0],temp_tr[1],temp_tr[4],0,temp_tr[2],temp_tr[7],temp_tr[5],temp_tr[21],
#                          temp_tr[22],temp_tr[23],temp_tr[24],temp_tr[25],temp_tr[26],temp_tr[27],temp_tr[28],temp_tr[29],
#                          temp_tr[30],temp_tr[31],temp_tr[32],temp_tr[33],temp_tr[20],temp_tr[3],temp_tr[8],temp_tr[9],
#                          temp_tr[10],temp_tr[11],temp_tr[12],temp_tr[13],temp_tr[14],temp_tr[15],temp_tr[16],temp_tr[17],
#                          temp_tr[18],temp_tr[19]])
#print(len(rows),len(rows)/2.0)

1828 914.0


In [36]:
#tourney_games_df = pd.DataFrame(rows, columns = ['season','daynum','team','win','opponent','num_ot','score',
#                                      'fgm','fga','fgm3','fga3','ftm','fta','or','dr','ast','to','stl','blk','fouls_given',
#                                      'fouls_received','opp_score','opp_fgm','opp_fga','opp_fgm3','opp_fga3','opp_ftm','opp_fta',
#                                      'opp_or','opp_dr','opp_ast','opp_to','opp_stl','opp_blk'])

In [38]:
#print(len(tourney_games_df))
#tourney_games_df.head()

1828


,season,daynum,team,win,opponent,num_ot,score,fgm,fga,fgm3,...,opp_fgm3,opp_fga3,opp_ftm,opp_fta,opp_or,opp_dr,opp_ast,opp_to,opp_stl,opp_blk
0,2003,134,1421,1,1411,1,92,32,69,11,...,12,31,14,31,17,28,16,15,5,0
1,2003,134,1411,0,1421,1,84,29,67,12,...,11,29,17,26,14,30,17,12,5,3
2,2003,136,1112,1,1436,0,80,31,66,7,...,4,16,7,7,8,26,12,17,10,3
3,2003,136,1436,0,1112,0,51,20,64,4,...,7,23,11,14,11,36,22,16,10,7
4,2003,136,1113,1,1272,0,84,31,59,6,...,7,28,14,21,20,22,11,12,2,5


In [39]:
#tourney_games = sqlalchemy.Table('tourney_games',meta,
#                                sqlalchemy.Column('season', sqlalchemy.String),
#                                sqlalchemy.Column('daynum', sqlalchemy.String),
#                                sqlalchemy.Column('team', sqlalchemy.String),
#                                sqlalchemy.Column('win', sqlalchemy.Integer),
#                                sqlalchemy.Column('opponent', sqlalchemy.String),
#                                sqlalchemy.Column('num_ot', sqlalchemy.Integer),
#                                sqlalchemy.Column('score', sqlalchemy.Integer),
#                                sqlalchemy.Column('fgm', sqlalchemy.Integer),
#                                sqlalchemy.Column('fga', sqlalchemy.Integer),
#                                sqlalchemy.Column('fgm3', sqlalchemy.Integer),
#                                sqlalchemy.Column('fga3', sqlalchemy.Integer),
#                                sqlalchemy.Column('ftm', sqlalchemy.Integer),
#                                sqlalchemy.Column('fta', sqlalchemy.Integer),
#                                sqlalchemy.Column('or', sqlalchemy.Integer),
#                                sqlalchemy.Column('dr', sqlalchemy.Integer),
#                                sqlalchemy.Column('ast', sqlalchemy.Integer),
#                                sqlalchemy.Column('to', sqlalchemy.Integer),
#                                sqlalchemy.Column('stl', sqlalchemy.Integer),
#                                sqlalchemy.Column('blk', sqlalchemy.Integer),
#                                sqlalchemy.Column('fouls_given', sqlalchemy.Integer),
#                                sqlalchemy.Column('fouls_received', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_score', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_fgm', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_fga', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_fgm3', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_fga3', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_ftm', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_fta', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_or', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_dr', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_ast', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_to', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_stl', sqlalchemy.Integer),
#                                sqlalchemy.Column('opp_blk', sqlalchemy.Integer))
#meta.create_all(engine)

In [40]:
#tourney_games_df.to_sql('tourney_games', engine, if_exists = 'append', index = False)

In [41]:
stmt = 'SELECT * FROM tourney_games LIMIT 5'

res_proxy = connection.execute(stmt)

query_df = pd.DataFrame(res_proxy.fetchall())
query_df.columns = res_proxy.keys()
query_df

,season,daynum,team,win,opponent,num_ot,score,fgm,fga,fgm3,...,opp_fgm3,opp_fga3,opp_ftm,opp_fta,opp_or,opp_dr,opp_ast,opp_to,opp_stl,opp_blk
0,2003,134,1421,1,1411,1,92,32,69,11,...,12,31,14,31,17,28,16,15,5,0
1,2003,134,1411,0,1421,1,84,29,67,12,...,11,29,17,26,14,30,17,12,5,3
2,2003,136,1112,1,1436,0,80,31,66,7,...,4,16,7,7,8,26,12,17,10,3
3,2003,136,1436,0,1112,0,51,20,64,4,...,7,23,11,14,11,36,22,16,10,7
4,2003,136,1113,1,1272,0,84,31,59,6,...,7,28,14,21,20,22,11,12,2,5


### Validate that the expected tables are in the database

In [43]:
#existing tables
print(engine.table_names())

['teams', 'tourney_seeds', 'team_conferences', 'tourney_geography', 'season_games', 'tourney_games']


### Disconnect From Database when done

In [44]:
connection.close()